In [7]:
import getpass

OPENAI_API_KEY = getpass.getpass("OpenAI API Key:")
ANTHROPIC_API_KEY = getpass.getpass("Anthropic API Key:")

In [20]:
from langchain_anthropic import ChatAnthropic
from langchain_community.chat_models import GigaChat
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI


@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' times 'y'."""
    return x * y


@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the 'y'."""
    return x**y


@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y


tools = [multiply, exponentiate, add]

gpt35 = ChatOpenAI(
    model="gpt-3.5-turbo-0125", temperature=0, api_key=OPENAI_API_KEY
).bind_tools(tools)

claude3 = ChatAnthropic(
    model="claude-3-sonnet-20240229", api_key=ANTHROPIC_API_KEY
).bind_tools(tools)

giga = GigaChat(
    profanity_check=False,
    verbose=True,
    verify_ssl_certs=False,
    timeout=30,
).bind_functions(tools)
llm_with_tools = gpt35.configurable_alternatives(
    ConfigurableField(id="llm"), default_key="gpt35", claude3=claude3, giga=giga
)

# LangGraph

In [18]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage
from langchain_core.runnables import RunnableLambda
from langgraph.graph import END, StateGraph


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


def should_continue(state):
    return "continue" if state["messages"][-1].tool_calls else "end"


def call_model(state, config):
    return {"messages": [llm_with_tools.invoke(state["messages"], config=config)]}


def _invoke_tool(tool_call):
    tool = {tool.name: tool for tool in tools}[tool_call["name"]]
    return ToolMessage(tool.invoke(tool_call["args"]), tool_call_id=tool_call["id"])


tool_executor = RunnableLambda(_invoke_tool)


def call_tools(state):
    last_message = state["messages"][-1]
    return {"messages": tool_executor.batch(last_message.tool_calls)}


workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tools)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)
workflow.add_edge("action", "agent")
graph = workflow.compile()

In [16]:
graph.invoke(
    {
        "messages": [
            HumanMessage(
                "Сколько будет 3 плюс 5 умножить на 2.743. Также сколько будет 17.24 - 918.1241"
            )
        ]
    }
)

{'messages': [HumanMessage(content='Сколько будет 3 плюс 5 умножить на 2.743. Также сколько будет 17.24 - 918.1241'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3MpNSPZi31IYDZ7Hh6vMm2wk', 'function': {'arguments': '{"x": 3, "y": 5}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_OAS60YQwngI6OZSuDkqEst3n', 'function': {'arguments': '{"x": 5, "y": 2.743}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_x3lqcMaIh9WCm2jofsgBES7Y', 'function': {'arguments': '{"x": 17.24, "y": -918.1241}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 178, 'total_tokens': 251}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-372aec6a-02c7-4d87-bcca-252ad18b5695-0', tool_calls=[{'name': 'add', 'args': {'x': 3, 'y': 5}, 'id': 'call_3MpNSPZi31IYDZ7Hh6vMm2wk'}, {'name': 'multiply', 'args': {'x': 5, 'y': 2.743}, 'id': 'ca

In [12]:
graph.invoke(
    {
        "messages": [
            HumanMessage(
                "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241"
            )
        ]
    },
    config={"configurable": {"llm": "claude3"}},
)

{'messages': [HumanMessage(content="what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241"),
  AIMessage(content=[{'text': 'Here are the calculations using the provided tools:', 'type': 'text'}, {'id': 'toolu_01VXpdRapN7CuSUxeptCmfY9', 'input': {'x': 3, 'y': 5}, 'name': 'add', 'type': 'tool_use'}], response_metadata={'id': 'msg_01GuHkqMXU7RASgcNQsFFeux', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 450, 'output_tokens': 78}}, id='run-b42b3d08-b302-4c37-8a52-b298f8b51834-0', tool_calls=[{'name': 'add', 'args': {'x': 3, 'y': 5}, 'id': 'toolu_01VXpdRapN7CuSUxeptCmfY9'}]),
  ToolMessage(content='8.0', tool_call_id='toolu_01VXpdRapN7CuSUxeptCmfY9'),
  AIMessage(content=[{'id': 'toolu_0122AU1x6hNTZ6Ljrp7JKdoP', 'input': {'x': 8.0, 'y': 2.743}, 'name': 'exponentiate', 'type': 'tool_use'}], response_metadata={'id': 'msg_01UdszDVXyrfL5Ab4Lin7ULH', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'tool_use', 'stop

In [21]:
graph.invoke(
    {
        "messages": [
            HumanMessage(
                "Сколько будет 3 плюс 5 умножить на 2.743. Также сколько будет 17.24 - 918.1241"
            )
        ]
    },
    config={"configurable": {"llm": "giga"}},
)

Giga request: {"model": null, "messages": [{"role": "user", "content": "Сколько будет 3 плюс 5 умножить на 2.743. Также сколько будет 17.24 - 918.1241", "function_call": null}], "temperature": null, "top_p": null, "n": null, "stream": null, "max_tokens": null, "repetition_penalty": null, "update_interval": null, "profanity_check": false, "function_call": null, "functions": null}
Giga response: Для решения первой задачи выполним действия в следующем порядке: сначала умножим 5 на 2.743, а затем к результату прибавим 3.

5 * 2.743 = 13.715
13.715 + 3 = 16.715

Таким образом, результат первой операции равен 16.715.

Для решения второй задачи также выполним действия в указанном порядке: сначала вычтем из 17.24 значение 918.1241, а затем округлим результат до двух знаков после запятой.

17.24 - 918.1241 = -900.8817

Округляем до двух знаков после запятой:
-900.88

Правильный ответ: 1) 16.715; 2) -900.88.


{'messages': [HumanMessage(content='Сколько будет 3 плюс 5 умножить на 2.743. Также сколько будет 17.24 - 918.1241'),
  AIMessage(content='Для решения первой задачи выполним действия в следующем порядке: сначала умножим 5 на 2.743, а затем к результату прибавим 3.\n\n5 * 2.743 = 13.715\n13.715 + 3 = 16.715\n\nТаким образом, результат первой операции равен 16.715.\n\nДля решения второй задачи также выполним действия в указанном порядке: сначала вычтем из 17.24 значение 918.1241, а затем округлим результат до двух знаков после запятой.\n\n17.24 - 918.1241 = -900.8817\n\nОкругляем до двух знаков после запятой:\n-900.88\n\nПравильный ответ: 1) 16.715; 2) -900.88.', response_metadata={'token_usage': Usage(prompt_tokens=47, completion_tokens=223, total_tokens=270), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'stop'}, id='run-e3d1ef55-a1be-46bf-9b4e-e607a8fba44f-0')]}